In [ ]:

import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np 
import numpy.random as npr
import scipy as sp
import scipy.linalg as sl
from scipy.fft import fft
from scipy.fft import ifft
plt.rcParams["animation.html"]="jshtml"


**Introduction**

Dans ce rapport nous essaierons de refaire l'experience des fentes de Young au travers d'un simulateur de Schrödinger
Nous commencerons d'abord par tester le simulateur lors de divers problème 1D, voir si les résultats sont bien cohérents avec les résultats historiques et théoriques

Puis nous regarderons des simulations de problème en 2D voir si nous avons les mêmes résultats ou des résultats semblables

Enfin nous regarderons les résultats pour l'experience des fentes de Young

Tout d'abord, nous allons regarder la boite à outils mise à notre disposition pour ce rapport. En dehors du simulateur de l'équation de schrödinger par méthode spectral qui a déjà été détailler, nous expliquerons quelques outils qui nous seront utiles lors de la suite de ce rapport.

**Les potentiels:**

Pour vérifier l'exactitude du simulateurs, nous regarderons les simulations de l'équation de schrödinger avec plusieurs potentiels différents:

- potentiel nul : cas simple qui nous permet de tester juste la partie avec le laplacien

- barrière de potentiel ou fonction créneau : nous permet de tester l'effet tunnel sur le simulateur

- puit de potentiel : nous permet de tester des états stationnaire dans un potentiel non nul et où l'on peut rationnaliser la périodicité induite par le simulateur

- potentiel quadratique : nous permet de tester des états stationnaire dans un potentiel smooth.


In [ ]:
#################  potentiel 1D ###############

V_0 =           lambda x: 0*x
V_barriere =    lambda x,L,V0,d:  (x>d)*(x<(d+L))*V0
V_puit =        lambda x,L,V0,d:  ((x<d) + (x>(d+L)))*V0
V_quad =        lambda x,x0,V0 :  V0*(x-x0)**2

################  potentiel 2D ################        

V_0_2D =           lambda x,y:0*x*y
## potentiels ~2D
V_barriere_2D = lambda x,y,L,V0,d:  (x>d)*(x<(d+L))*V0
V_tube_2D =     lambda x,y,L,V0,d:  ((x<d) + (x>(d+L)))*V0
V_quad_2D =     lambda x,y,x0,y0,V0 :  V0*(x-x0)**2
## potentiels radiaux
V_couronne_2D = lambda x,y,x0,y0,L,V0,d:    V_barriere(np.sqrt((x-x0)**2 + (y-y0)**2),L,V0,d)
V_puit_2D  =    lambda x,y,x0,y0,L,V0,d:    V_puit(np.sqrt((x-x0)**2 + (y-y0)**2), L,V0,d)
V_quad_2D =     lambda x,y,x0,y0,V0:        V_quad(np.sqrt((x-x0)**2 + (y-y0)**2), 0,V0)  

**Jauges**

Les "Jauges" seront des outils pour jauger différentes quantités de la simulation et/ou voir  si elles se conservent comme en théorie. Parmis les Jauges, on note :

- la jauge de probabilité pour vérifier la conservation de la probabilité et voir les pertes dues aux transmission/effets tunnnels.

- la Jauge d'énergie pour regarder la conservation de l'énergie

- le Jauge max pour voir la valeur du pic de probabilité qui nous permettra de regarder les coefficients de transmission/ reflexion

In [ ]:

def Norme_psi(psi,Nx,L):
    C_psi = fft(psi)
    norm = np.abs(C_psi)**2 *(np.fft.fftfreq(Nx,L/Nx)**2 +1)
    return sum(norm)


def Concentration_de_masse(psi,Nx,L,a,b):
    l = np.abs(b-a)
    an = int((np.abs(a+L)/(2*L))*Nx) ; bn = int((np.abs(b+L)/(2*L))*Nx)
    psi_l= np.zeros(Nx)
    psi_l[an:bn] = psi[an:bn]
    return 2*Norme_psi(psi_l,Nx,L)

def Energie(psi,V,Nx,L,t):
    I = np.linspace(-L,L,Nx)
    C_psi = fft(psi); V_psi= fft(V(I,t)*psi)
    energ = 0.5*np.abs(C_psi)*(np.fft.fftfreq(Nx,L/Nx)**2) + np.abs(V_psi)
    return sum(energ)


def Jauges(psi,Nx,L,t, V_fun=(lambda x,t: 0)):
    print("norme de psi sur [-L,L] = ", Norme_psi(psi,Nx,L))
    print("Energie de psi = ", Energie(psi,V_fun,Nx,L,t))
    print("============================== ",t," ===========================")


**Régularisation des potentiels**

Un problème que l'on peut vouloir étudié est l'erreur et la dispersion numérique induite par la discontinuité des potentiels. 

Pour regarder leurs effets, nous avons mis au point des potentiels similaire régularisé avec des splines $C^\infty$

Les potentiels 2D équivalents aux potentiels sont réguliers car ce sont des potentiels radiaux ou des potentiels 1D auquel on a rajouté une dimension. On peut donc appliquée un spline 1D sans problème et rajouter de la régularité de la même manière que si il s'agissait d'un potentiel 1D.

In [ ]:
def splin_exp(x):
    if(x<0): return 0
    if(x>1): return 1
    return np.exp(-1/x) / (np.exp(-1/x)+ np.exp(-1/(1-x)))

## raccorde [(p1,a),(p2,b)]
raccord = lambda x,p1,p2,a,b: (1-splin_exp((x-a)/(b-a)))*p1 + splin_exp((x-a)/(b-a))*p2 


Vr_barriere =    lambda x,L,V0,d,epsi:  V_barriere(x,L,V0,d) * splin_exp((x-(d-epsi))/(2*epsi)) *splin_exp((x-(d+L-epsi))/(-2*epsi))
Vr_puit =        lambda x,L,V0,d,epsi:  V_puit(x,L,V0,d) *     splin_exp((x-(d-epsi))/(2*epsi)) *splin_exp((x-(d+L-epsi))/(-2*epsi))

################  potentiel 2D ################        
ray = lambda x,y: np.sqrt(x**2 +y**2)
## potentiels ~2D
Vr_barriere_2D = lambda x,y,L,V0,d,epsi:  V_barriere_2D(x,y,L,V0,d)*splin_exp((x-(d-epsi))/(2*epsi))
Vr_tube_2D =     lambda x,y,L,V0,d,epsi:  V_tube_2D(x,y,L,V0,d) *   splin_exp((x-(d-epsi))/(2*epsi))
## potentiels radiaux
Vr_couronne_2D = lambda x,y,x0,y0,L,V0,d,epsi: V_couronne_2D(x,y,x0,y0,L,V0,d)*splin_exp((ray(x-x0,y-y0)-(d-epsi))/(2*epsi)) *splin_exp((ray(x-x0,y-y0)-(d+L-epsi))/(2*epsi))
Vr_puit_2D  =    lambda x,y,x0,y0,L,V0,d,epsi: V_puit_2D(x,y,x0,y0,L,V0,d)    *splin_exp((ray(x-x0,y-y0)-(d-epsi))/(2*epsi))

**Autre fonctions utiles**


- eig_function : état stationnaire pour le potentiel nul de norme $A$ que l'on peut modulé par une phase $\phi$

- weyl_func :    état quasi stationnaire, on peut le considérer comme un élément d'une suite de Weyl. Sa fonction de densité est celle d'un état stationnaire que l'on multiplie par une gaussienne centrée en $x_0$ de variance $\sigma$. 

- projection_red et projection_red_2D: deux fonctions avec le même but, réduire la taille de stockage mémoire de la solution. elle réduisent d'un facteur red_x, ( ou red_x*red_y) le poid mémoire de la solution.
Cela permet de réduire le poids de la solution sans impacter son erreur

In [ ]:
def eig_func(x,k,A=1.,phi=0+0j):
    return A*np.exp(1j*x*k)*np.exp(1j*phi)

def weyl_func(x,k,A=1.,phi=0+0j,x0=0.,sig=1.):
    return np.exp(-(x-x0)**2/(sig)**2) * eig_func(x,k,A,phi)

def Projection_red (Nx,psi,red_x):
    sol_red = np.zeros(int(Nx/red_x), dtype="complex")
    for i in range(0,Nx,red_x):
        sol_red[int(i/red_x)] = psi[i]
    return sol_red

def Projection_red_2D (Nx,Ny,psi,red_x,red_y):
    sol_red = np.zeros((int(Nx/red_x),int(Ny/red_y)), dtype="complex")
    for i in range(0,Nx,red_x):
        for j in range(0,Ny,red_y):
            sol_red[int(i/red_x),int(j/red_y)] = psi[i,j]
    return sol_red

**Quelques commentaire sur le simulateur de schrödinger**



In [ ]:
def dynamics_fft_diss(psi0_fun=(lambda x: np.exp(-x**2)), V_fun=(lambda x,t: 0.), L=10., Nx=100, T=4., Nt=100, reduction=1, show_phi=False, savefile=1):

    dt = T/Nt; dx = L/Nx
    # Kinetic = Kin(Nx)
    Kinetic = (0.5*(2*np.pi/L)**2) *np.fft.fftfreq(Nx, dx)*np.fft.fftfreq(Nx, dx)
    I = np.linspace(-L, L,Nx,endpoint=False)
    

    Psi_temp =np.zeros((Nx,2), dtype="complex"); Phi_temp = np.zeros((Nx), dtype="complex")
    Psi_temp[:,0]=psi0_fun(I)
    
    Psi_T = np.zeros((int(Nx/reduction),Nt), dtype="complex")

    if(not show_phi) : Psi_T[:,0]= Projection_red(Nx,Psi_temp[:,0],reduction)
  
    for i in range(1,Nt*savefile):
        ti = dt*i
        Phi_temp[:]= (np.exp(-1j*Kinetic*dt)) * fft(np.exp(-1j*V_fun(I,ti)*dt) * Psi_temp[:,0])
        Psi_temp[:,1]= ifft(Phi_temp[:])
        
        if(i%savefile==0) : 
            if(not show_phi) :  Psi_T[:,int(i/savefile)]  = Projection_red(Nx,Psi_temp[:,1],reduction); 
            else :              Psi_T[:,int(i/savefile)]  = Projection_red(Nx,Phi_temp[:],reduction); 
        
        Psi_temp[:,0] = Psi_temp[:,1]

    return Psi_T

In [ ]:

def plot_psi(psi, duration=10, frames_per_second=30, L=10, show_potential=False,  V_fun=(lambda x,t: 0)):
    
    fig, ax = plt.subplots()
    t_data = np.linspace(0, 1, np.size(psi, 1)) # 1 is arbitrary here
    x_data = np.linspace(-L,L,  np.size(psi,0))
    # set the min and maximum values of the plot, to scale the axis
    m = min(0, np.min(np.real(psi)), np.min(np.imag(psi)))
    M = np.max(np.abs(psi))
    
    # set the axis once and for all
    ax.set(xlim=[-L,L], ylim=[m,M], xlabel='x', ylabel='psi')
    
    # dummy plots, to update during the animation
    real_plot = ax.plot(x_data, np.real(psi[:, 0]), label='Real')[0]
    imag_plot = ax.plot(x_data, np.imag(psi[:, 0]), label='Imag')[0]
    abs_plot  = ax.plot(x_data, np.abs(psi[:, 0]), label='Abs')[0]
    if(show_potential):V_plot  =   ax.plot(x_data, V_fun(x_data,0), label='V')[0]
    ax.legend()

    # define update function as an internal function (that can access the variables defined before)
    # will be called with frame=0...(duration*frames_per_second)-1
    def update(frame):
        IPython.display.clear_output(wait=True)
        print(frame)
        # get the data by linear interpolation
        t = frame / (duration * frames_per_second)
        psi_t = np.array([np.interp(t, t_data, psi[i, :]) for i in range(np.size(psi,0))])
        # update the plots
        real_plot.set_ydata(np.real(psi_t))
        imag_plot.set_ydata(np.imag(psi_t))
        abs_plot.set_ydata(np.abs(psi_t))

    ani = animation.FuncAnimation(fig=fig, func=update, frames=duration*frames_per_second, interval=1000/frames_per_second)
    return ani


**Problème 1D**
- Pour le potentiel nul, on s'attends à voir que si on raccorde les état stationnaire pour qu'ils deviennent périodique sur la zone de simulation, alors ils ne devraient être.. stationnaire.
- On remarque que en plus de conserver la norme L2, il conserve l'énergie.

In [ ]:
## parametre simulation
Nx = 1000; Nt=5000; L=1
## parametre physique
k=1; T=1

I = np.linspace(-L,L,Nx)

## on normalise la fonction
C=1/np.sqrt(Norme_psi(eig_func(I,2*np.pi*L*k,1),Nx,L))
psi0 =lambda x: eig_func(x,2*np.pi*L*k,C)
V = lambda x,t: V_0(x) 


psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt)
for i in range(0,Nt,int(np.floor(Nt/10))):
    Jauges(psi[:,i],Nx,L,i/Nt)

In [ ]:

anime = plot_psi(psi,L=L, duration=10, frames_per_second=60,show_potential=True,V_fun=V)
plt.close()
anime

**barrière de potentielle**

- on remarque que l'on conserve la probabilité au cours de la simulation. On ne conserve cependant pas l'energie. Cela peut être due aux phénomènes de réflexion-transmission induit par le potentiel. 
- on constate aussi que les coefficients de réflexion-transmission sont bien en accord avec la théorie: 

\begin{equation}
    T = \frac{1}{1+\frac{4E}{V_0}\left(1-\frac{E}{V_0}\right) \sinh(l\sqrt{2(V_0-E)})^2} 
\end{equation}

- si l'on pose $N_t<N_x$ alors on obtiens des phénomène de dispersion numérique lorsque l'onde intéragit avec la barrière de potentiel.

In [ ]:
## parametre simulation
Nx = 4000; Nt=10000; L=10
## parametre physique
k=20+0j; T=4; d=3; l=0.1; V0=50

I = np.linspace(-L,L,Nx)

## on normalise la fonction
C= 1/ np.sqrt(Norme_psi(weyl_func(I,k,1,0,0,0.1),Nx,L))

psi0 =lambda x: weyl_func(x,k,C,0,0,0.1)
V = lambda x,t: V_barriere(x,l,V0,d) 

K= np.sqrt(2*(V0-0.5*(k**2))); K2= K**2; k2=-k**2

Trans = float( 4*(K2*k2) /( (K2+k2)**2 *(np.sinh(K*l))**2 +4*K2*k2))
 
print(Trans, V0, 0.5*K2)

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, savefile=3)

### analyse des résultats
for i in range(0,Nt,int(np.floor(Nt/10))):
    t = T*i/Nt
    print("============================== ",t," ===========================")
    print(Concentration_de_masse(psi[:,i],Nx,L,-L,d)," masse avant l'interface")
    # print(Concentration_de_masse(psi[:,i],Nx,L,d,d+l)," masse dans l'interface")
    print(Concentration_de_masse(psi[:,i],Nx,L,d+l,L)," masse après l'interface")
    # print("norme de psi sur [-L,L] = ", Norme_psi(psi[:,i],Nx,L))
    print("Energie de psi = ", Energie(psi[:,i],V,Nx,L,t))


In [ ]:
anime = plot_psi(psi,L=L, duration=10, frames_per_second=10,show_potential=True,V_fun=V)
plt.close()
anime